# CMA --> ttv
cma出力ファイルをttv形式に変換します。

In [1]:
import csv, datetime
import glob
import util_csv
import util_time
import editrow
# データファイル一覧の読み込み
files=glob.glob("./data/*.csv")

In [3]:
# 本番用test
file_name = './data/33_sample.csv'
file_name_list = file_name.split('/')
result_dir_name = './result/'
result_file_name = result_dir_name + file_name_list[2]
# './result/33_sample.csv'
assert result_file_name ==  './result/33_sample.csv'
# ファイルのread
file = util_csv.read_csv(file_name)

# ファイルの分割、書き込み
separate_file = editrow.separate_row(file, '。')
print('ファイル名： '+ file_name)
print('分割前： ' + str(len(file)))
print('分割後： ' + str(len(separate_file)))

# タイムスタンプ形式への更新
next_times = util_time.get_next_times(separate_file)
vtt_times = util_time.get_vtt_times(next_times)
assert(len(separate_file)==len(vtt_times))

# ファイルの書き込み
with open(result_file_name, 'w')as o:
    writer = csv.writer(o, delimiter=',')
    for i, row in enumerate(separate_file):
        writer.writerow([vtt_times[i], row[1], row[2]])


ファイル名： ./data/33_sample.csv
分割前： 76
分割後： 179
total179


In [5]:
f = './result/33_sample.csv'
file = util_csv.read_csv(f)
with open('./result/33_sample.vtt', 'w') as o:
    o.write('WEBVTT\n\n')
    for i, row in enumerate(file):
        o.write(row[0] + '\n' + row[2] + '\n\n')
        
        
    

In [10]:
first_date = datetime.datetime.strptime('2020/02/02 00:03:30', '%Y/%m/%d %H:%M:%S')
second_date = datetime.datetime.strptime('2020/02/02 00:01:00', '%Y/%m/%d %H:%M:%S')

diff = first_date - second_date
print(diff)
print(diff.seconds)

0:02:30
150


In [13]:

def get_h_m_s(td):
    m, s = divmod(td.seconds, 60)
    h, m = divmod(m, 60)
    if len(str(h))==1:
        h = '0'+str(h)
    else:
        h = str(h)
    if len(str(m))==1:
        m = '0'+str(m)
    else:
        s = str(m)
    if len(str(s))==1:
        s = '0'+str(s)
    else:
        s = str(s)
    return h, m, s

In [56]:
def get_times2(start_date, check_date, current_date, length, word_nums):
    # 差分から推定時間をリストにして返却
    diff_time = current_date - check_date #差分時間
    # TODO : 一度secondsで計算後、mm:ssに戻す
    diff_seconds = diff_time.seconds # 時間、分含めた総合秒数
    diff_time = int(get_h_m_s(diff_time)[2])
    check_date_ss = check_date.second # 
    print(diff_seconds, check_date_ss)
    
    # args: ex 01:30:00 , 01:30:20, 3
    # return : [01:30:00, 01:30:05, 01:30:15]
    percentages = list(map(lambda i:i/sum(word_nums), word_nums))
    time_percentages = list(map(lambda i:i*diff_seconds, percentages))
    # 整数への切り上げ
    time_percentages = list(map(lambda i:int(Decimal(str(i)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)),time_percentages))
    if (sum(time_percentages) != diff_seconds):
        time_percentages[-1] = diff_seconds - sum(time_percentages[:-1])
    print(time_percentages)
    assert sum(time_percentages) == diff_seconds
    
    # total_time = 0
    # [24, 77, 49]
    # ーー＞ [00:00:00, 00:00:24, 00:00:101]
    # 初期値は１つ前の時間（＝check_date)
    total_time = 0
    times = []
    for i, time in enumerate(time_percentages):
        if (i==0):
            # 初期値＝check_timeが入る
            pass
        else:
            # 合計した値の秒数となる
            total_time += time_percentages[i-1]
        
        # check_dateにtotal_timeを足した日付が発話開始時間
        # 開始日付からの経過時間を算出してvtt形式対応する
        td = (check_date + datetime.timedelta(seconds=total_time)) - start_date
        hh, mm, ss = get_h_m_s(td)
        time = hh +':'+ mm +':'+ ss +'.000'
        times.append(time)

    return times


In [54]:
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
first_date = datetime.datetime.strptime('2020/02/02 00:07:30', '%Y/%m/%d %H:%M:%S')
second_date = datetime.datetime.strptime('2020/02/02 00:06:00', '%Y/%m/%d %H:%M:%S')
start_date = datetime.datetime.strptime('2020/02/02 00:05:00', '%Y/%m/%d %H:%M:%S')
third = first_date + datetime.timedelta(seconds=10)
print(third)
times = get_times2(start_date, second_date, first_date, 3, [10,32,20])
print(times)

2020-02-02 00:07:40
90 0
[15, 46, 29]
['00:01:00.000', '00:01:15.000', '00:02:01.000']
